In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
# import dataframe_image as dfi
# import imgkit as img 




In [2]:
df_edited = pd.read_csv('./data/clean_df_final.csv', index_col=0 )

F:\Anaconda\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (7,8,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Reset Display or Max Display setting here

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#TO SEE EVERYTHING

In [4]:
pd.reset_option('all')
#RUN THIS WHEN DONE SEEING EVERYTHING PLS


: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



F:\Anaconda\envs\learn-env\lib\site-packages\pandas\_config\config.py:622: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


# Creating new DF of only post 2010

In [5]:
df_edited2 = df_edited[df_edited['year']>2010].copy()

In [6]:
df_edited2['counter'].sum()

19514

# Fatality Cleaning

In [7]:
df_edited2['total_pop'] = (df_edited2['Total.Fatal.Injuries'] + df_edited2['Total.Minor.Injuries'] + df_edited2['Total.Serious.Injuries'] + df_edited2['Total.Uninjured'])

#Creating total_pop col

In [8]:
#GROUP BY ENGINE AND MAKE

In [9]:
sum_engine_make = df_edited2.groupby(['Make', 'Number.of.Engines']).sum().sort_values(by='total_pop', ascending = False)

In [10]:
sum_engine_make['fatality_perc'] = (sum_engine_make['Total.Fatal.Injuries']/sum_engine_make['total_pop']) * 100
sum_engine_make['injury_perc'] = ((sum_engine_make['Total.Serious.Injuries']+sum_engine_make['Total.Minor.Injuries'])/sum_engine_make['total_pop']) * 100
sum_engine_make['uninjured_perc'] = (sum_engine_make['Total.Uninjured']/sum_engine_make['total_pop']) * 100
sum_engine_make.reset_index(inplace = True)

In [11]:
#GROUP BY MAKE

In [12]:
sum_make = df_edited2.groupby(['Make']).sum().sort_values(by='total_pop', ascending = False)

In [13]:
sum_make['fatality_perc'] = (sum_make['Total.Fatal.Injuries']/sum_make['total_pop']) * 100
sum_make['injury_perc'] = ((sum_make['Total.Serious.Injuries']+sum_make['Total.Minor.Injuries'])/sum_make['total_pop']) * 100
sum_make['uninjured_perc'] = (sum_make['Total.Uninjured']/sum_make['total_pop']) * 100
sum_make.reset_index(inplace = True)

In [14]:
#GROUP BY ENGINE NUM
sum_engine = df_edited2.groupby(['Number.of.Engines']).sum().sort_values(by='total_pop', ascending = False)

In [15]:
sum_engine['fatality_perc'] = (sum_engine['Total.Fatal.Injuries']/sum_engine['total_pop']) * 100
sum_engine['injury_perc'] = ((sum_engine['Total.Serious.Injuries']+sum_engine['Total.Minor.Injuries'])/sum_engine['total_pop']) * 100
sum_engine['uninjured_perc'] = (sum_engine['Total.Uninjured']/sum_engine['total_pop']) * 100
sum_engine.reset_index(inplace = True)

# Fatality Analyzing

In [16]:
new_sum_engine = sum_engine[sum_engine['counter'] >= 0]
new_sum_engine

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,year,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
0,2.0,1466.0,916.0,499.0,54786.0,4239798,2103,57667.0,2.542182,2.453743,95.004075
1,-99.0,4873.0,960.0,717.0,45604.0,6546690,3245,52154.0,9.343483,3.215477,87.441040
2,1.0,4328.0,3657.0,2443.0,13784.0,27996212,13886,24212.0,17.875434,25.194119,56.930448
3,4.0,24.0,9.0,15.0,3418.0,108811,54,3466.0,0.692441,0.692441,98.615118
4,0.0,38.0,66.0,79.0,341.0,423324,210,524.0,7.251908,27.671756,65.076336
5,3.0,79.0,0.0,4.0,86.0,24190,12,169.0,46.745562,2.366864,50.887574
6,8.0,0.0,21.0,3.0,11.0,6059,3,35.0,0.000000,68.571429,31.428571
7,6.0,0.0,0.0,0.0,0.0,2022,1,0.0,NaN,NaN,NaN


In [17]:
new_sum_engine = new_sum_engine.drop(['year'], axis=1)

In [18]:
new_sum_engine.sort_values(by='uninjured_perc', ascending=False, inplace=True)

In [19]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

styled_table = (new_sum_engine.style
    .format({'Total.Fatal.Injuries': '{:,.0f}', 'Total.Serious.Injuries': '{:,.0f}', 'Total.Minor.Injuries': '{:,.0f}', 'Total.Uninjured': '{:,.0f}', 'total_pop': '{:,.0f}', 'fatality_perc': '{:.2f}%', 'injury_perc': '{:.2f}%', 'uninjured_perc': '{:.2f}%'})
    .format({'Number.of.Engines': '{:.0f}'})
    .applymap(lambda x: 'background-color: lightgreen' if isinstance(x, float) and x > 90.00 else ('background-color: lightcoral' if x == -99 else ''), subset=pd.IndexSlice[:, ['uninjured_perc', 'Number.of.Engines']])
    .set_properties(**{'font-size': '12pt', 'font-family': 'Calibri', 'text-align': 'center', 'border': '1px solid black'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#337ab7'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid grey')]}])
                        .hide_index())
    
display(styled_table)






Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
4,24,9,15,"3,418",54,"3,466",0.69%,0.69%,98.62%
2,"1,466",916,499,"54,786",2103,"57,667",2.54%,2.45%,95.00%
-99,"4,873",960,717,"45,604",3245,"52,154",9.34%,3.22%,87.44%
0,38,66,79,341,210,524,7.25%,27.67%,65.08%
1,"4,328","3,657","2,443","13,784",13886,"24,212",17.88%,25.19%,56.93%
3,79,0,4,86,12,169,46.75%,2.37%,50.89%
8,0,21,3,11,3,35,0.00%,68.57%,31.43%
6,0,0,0,0,1,0,nan%,nan%,nan%


In [20]:
new_sum_engine.drop([1], axis = 0, inplace = True)

In [21]:
new_sum_engine.sort_values(by = 'uninjured_perc', ascending = False)

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
3,4.0,24.0,9.0,15.0,3418.0,54,3466.0,0.692441,0.692441,98.615118
0,2.0,1466.0,916.0,499.0,54786.0,2103,57667.0,2.542182,2.453743,95.004075
4,0.0,38.0,66.0,79.0,341.0,210,524.0,7.251908,27.671756,65.076336
2,1.0,4328.0,3657.0,2443.0,13784.0,13886,24212.0,17.875434,25.194119,56.930448
5,3.0,79.0,0.0,4.0,86.0,12,169.0,46.745562,2.366864,50.887574
6,8.0,0.0,21.0,3.0,11.0,3,35.0,0.000000,68.571429,31.428571
7,6.0,0.0,0.0,0.0,0.0,1,0.0,NaN,NaN,NaN


In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

styled_table = (new_sum_engine.style
    .format({'Total.Fatal.Injuries': '{:,.0f}', 'Total.Serious.Injuries': '{:,.0f}', 'Total.Minor.Injuries': '{:,.0f}', 'Total.Uninjured': '{:,.0f}', 'total_pop': '{:,.0f}', 'fatality_perc': '{:.2f}%', 'injury_perc': '{:.2f}%', 'uninjured_perc': '{:.2f}%'})
    .format({'Number.of.Engines': '{:.0f}'})
    .applymap(lambda x: 'background-color: lightgreen' if isinstance(x, float) and x > 90.00 else ('background-color: lightcoral' if x == -99 else ''), subset=pd.IndexSlice[:, ['uninjured_perc', 'Number.of.Engines']])
    .set_properties(**{'font-size': '12pt', 'font-family': 'Calibri', 'text-align': 'center', 'border': '1px solid black'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#337ab7'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid grey')]}])
                        .hide_index())
    
display(styled_table)

Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
4,24,9,15,"3,418",54,"3,466",0.69%,0.69%,98.62%
2,"1,466",916,499,"54,786",2103,"57,667",2.54%,2.45%,95.00%
0,38,66,79,341,210,524,7.25%,27.67%,65.08%
1,"4,328","3,657","2,443","13,784",13886,"24,212",17.88%,25.19%,56.93%
3,79,0,4,86,12,169,46.75%,2.37%,50.89%
8,0,21,3,11,3,35,0.00%,68.57%,31.43%
6,0,0,0,0,1,0,nan%,nan%,nan%


In [23]:
##CONCLUSION: 3 4 and 2 are safest
## 0 and 1 are not safest

In [24]:
new_sum_engine_make = sum_engine_make[sum_engine_make['counter'] >= 25]

In [25]:
new_sum_engine_make = new_sum_engine_make[new_sum_engine_make['Number.of.Engines']!= -99]

In [26]:
new_sum_engine_make = new_sum_engine_make.sort_values(by = 'uninjured_perc', ascending = False).head(10)

In [27]:
new_sum_engine_make = new_sum_engine_make.drop(['year'], axis=1)

In [28]:
new_sum_engine_make.drop(['Total.Fatal.Injuries', 'Total.Serious.Injuries','Total.Minor.Injuries', 'Total.Uninjured' ], axis=1, inplace=True)

In [29]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

styled_table = (new_sum_engine_make.style
    .format({'Total.Fatal.Injuries': '{:,.0f}', 'Total.Serious.Injuries': '{:,.0f}', 'Total.Minor.Injuries': '{:,.0f}', 'Total.Uninjured': '{:,.0f}', 'total_pop': '{:,.0f}', 'fatality_perc': '{:.2f}%', 'injury_perc': '{:.2f}%', 'uninjured_perc': '{:.2f}%'})
    .format({'Number.of.Engines': '{:.0f}'})
    .applymap(lambda x: 'background-color: lightgreen' if isinstance(x, float) and x > 90.00 else ('background-color: lightcoral' if x == -99 else ''), subset=pd.IndexSlice[:, ['uninjured_perc', 'Number.of.Engines']])
    .set_properties(**{'font-size': '12pt', 'font-family': 'Calibri', 'text-align': 'center', 'border': '1px solid black'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#337ab7'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid grey')]}])
                        .hide_index())
    
display(styled_table)

Make,Number.of.Engines,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
Airbus,2,127,"12,202",0.02%,1.11%,98.87%
Bombardier,2,62,"2,500",0.24%,1.16%,98.60%
Boeing,4,30,"2,286",0.96%,0.70%,98.34%
Embraer,2,56,"2,038",0.69%,1.23%,98.09%
Boeing,2,320,"32,323",0.46%,1.81%,97.73%
Maule,1,116,176,6.25%,18.18%,75.57%
Boeing,1,69,124,6.45%,23.39%,70.16%
Aero Commander,1,26,36,8.33%,25.00%,66.67%
Stinson,1,95,166,2.41%,31.33%,66.27%
Schweizer,1,70,102,7.84%,26.47%,65.69%


In [30]:
new_sum_engine_make2 = new_sum_engine_make[new_sum_engine_make['Number.of.Engines'] == 2].sort_values(by = 'uninjured_perc', ascending = False)

#deeper look at 2 engines

In [31]:
new_sum_engine_make2

,Make,Number.of.Engines,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
2,Airbus,2.0,127,12202.0,0.016391,1.114571,98.869038
6,Bombardier,2.0,62,2500.0,0.240000,1.160000,98.600000
8,Embraer,2.0,56,2038.0,0.686948,1.226693,98.086359
1,Boeing,2.0,320,32323.0,0.457878,1.809857,97.732265


In [32]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

styled_table = (new_sum_engine_make2.style
    .format({'Total.Fatal.Injuries': '{:,.0f}', 'Total.Serious.Injuries': '{:,.0f}', 'Total.Minor.Injuries': '{:,.0f}', 'Total.Uninjured': '{:,.0f}', 'total_pop': '{:,.0f}', 'fatality_perc': '{:.2f}%', 'injury_perc': '{:.2f}%', 'uninjured_perc': '{:.2f}%'})
    .format({'Number.of.Engines': '{:.0f}'})
    .applymap(lambda x: 'background-color: lightgreen' if isinstance(x, float) and x > 90.00 else ('background-color: lightcoral' if x == -99 else ''), subset=pd.IndexSlice[:, ['uninjured_perc', 'Number.of.Engines']])
    .set_properties(**{'font-size': '12pt', 'font-family': 'Calibri', 'text-align': 'center', 'border': '1px solid black'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#337ab7'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid grey')]}])
                        .hide_index())
    
display(styled_table)

Make,Number.of.Engines,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
Airbus,2,127,"12,202",0.02%,1.11%,98.87%
Bombardier,2,62,"2,500",0.24%,1.16%,98.60%
Embraer,2,56,"2,038",0.69%,1.23%,98.09%
Boeing,2,320,"32,323",0.46%,1.81%,97.73%


# Deeper analyzing into 2 engine Airbus and Boeing (maybe cut)

In [33]:
airbus_boeing_2eng = df_edited2.groupby(['Make','Number.of.Engines','Engine.Type']).sum()

In [34]:
airbus_boeing_2eng = airbus_boeing_2eng[airbus_boeing_2eng['counter'] >= 75]

In [35]:
airbus_boeing_2eng['fatality_perc'] = (airbus_boeing_2eng['Total.Fatal.Injuries']/airbus_boeing_2eng['total_pop']) * 100
airbus_boeing_2eng['injury_perc'] = ((airbus_boeing_2eng['Total.Serious.Injuries']+airbus_boeing_2eng['Total.Minor.Injuries'])/airbus_boeing_2eng['total_pop']) * 100
airbus_boeing_2eng['uninjured_perc'] = (airbus_boeing_2eng['Total.Uninjured']/airbus_boeing_2eng['total_pop']) * 100
airbus_boeing_2eng.reset_index(inplace = True)

In [36]:
airbus_boeing_2eng = airbus_boeing_2eng[airbus_boeing_2eng['Number.of.Engines']!=-99]

In [37]:
airbus_boeing_2eng = airbus_boeing_2eng.sort_values('uninjured_perc', ascending = False).head(20)

In [38]:
airbus_boeing_2eng

,Make,Number.of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,year,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
3,Airbus,2.0,Turbo Fan,2.0,86.0,22.0,8887.0,151188,75,8997.0,0.022230,1.200400,98.777370
15,Boeing,2.0,Unknown,134.0,44.0,34.0,10534.0,266253,132,10746.0,1.246976,0.725851,98.027173
14,Boeing,2.0,Turbo Fan,7.0,382.0,115.0,19611.0,358711,178,20115.0,0.034800,2.470793,97.494407
26,Maule,1.0,Reciprocating,9.0,22.0,5.0,116.0,203567,101,152.0,5.921053,17.763158,76.315789
9,Beech,2.0,Turbo Prop,70.0,18.0,21.0,290.0,213616,106,399.0,17.543860,9.774436,72.681704
17,Cessna,1.0,Reciprocating,711.0,825.0,396.0,3982.0,6571033,3260,5914.0,12.022320,20.645925,67.331755
35,Stinson,1.0,Reciprocating,4.0,35.0,12.0,98.0,175390,87,149.0,2.684564,31.543624,65.771812
18,Cessna,1.0,Turbo Prop,58.0,39.0,27.0,236.0,167321,83,360.0,16.111111,18.333333,65.555556
21,Champion,1.0,Reciprocating,23.0,29.0,17.0,125.0,252027,125,194.0,11.855670,23.711340,64.432990
0,Aeronca,1.0,Reciprocating,16.0,24.0,17.0,96.0,213642,106,153.0,10.457516,26.797386,62.745098


In [39]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

styled_table = (airbus_boeing_2eng.style
    .format({'Total.Fatal.Injuries': '{:,.0f}', 'Total.Serious.Injuries': '{:,.0f}', 'Total.Minor.Injuries': '{:,.0f}', 'Total.Uninjured': '{:,.0f}', 'total_pop': '{:,.0f}', 'fatality_perc': '{:.2f}%', 'injury_perc': '{:.2f}%', 'uninjured_perc': '{:.2f}%'})
    .format({'Number.of.Engines': '{:.0f}'})
    .applymap(lambda x: 'background-color: lightgreen' if isinstance(x, float) and x > 90.00 else ('background-color: lightcoral' if x == -99 else ''), subset=pd.IndexSlice[:, ['uninjured_perc', 'Number.of.Engines']])
    .set_properties(**{'font-size': '12pt', 'font-family': 'Calibri', 'text-align': 'center', 'border': '1px solid black'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#337ab7'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid grey')]}])
                        .hide_index())
    
display(styled_table)

Make,Number.of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,year,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
Airbus,2,Turbo Fan,2,86,22,"8,887",151188,75,"8,997",0.02%,1.20%,98.78%
Boeing,2,Unknown,134,44,34,"10,534",266253,132,"10,746",1.25%,0.73%,98.03%
Boeing,2,Turbo Fan,7,382,115,"19,611",358711,178,"20,115",0.03%,2.47%,97.49%
Maule,1,Reciprocating,9,22,5,116,203567,101,152,5.92%,17.76%,76.32%
Beech,2,Turbo Prop,70,18,21,290,213616,106,399,17.54%,9.77%,72.68%
Cessna,1,Reciprocating,711,825,396,"3,982",6571033,3260,"5,914",12.02%,20.65%,67.33%
Stinson,1,Reciprocating,4,35,12,98,175390,87,149,2.68%,31.54%,65.77%
Cessna,1,Turbo Prop,58,39,27,236,167321,83,360,16.11%,18.33%,65.56%
Champion,1,Reciprocating,23,29,17,125,252027,125,194,11.86%,23.71%,64.43%
Aeronca,1,Reciprocating,16,24,17,96,213642,106,153,10.46%,26.80%,62.75%


In [40]:
airbus_boeing_2eng[airbus_boeing_2eng['Make']=='Airbus']

,Make,Number.of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,year,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
3,Airbus,2.0,Turbo Fan,2.0,86.0,22.0,8887.0,151188,75,8997.0,0.02223,1.2004,98.77737


# Grouping by Make, Engine Num, Engine Type, and Purpose

In [41]:
df_edited2['Purpose.of.flight'] = df_edited2['Purpose.of.flight'].fillna('Unknown')

In [42]:
purpose_df = df_edited2.groupby(['Make','Number.of.Engines','Engine.Type', 'Purpose.of.flight']).sum()

In [43]:
purpose_df = purpose_df[purpose_df['counter'] >= 25]

In [44]:
purpose_df['fatality_perc'] = (purpose_df['Total.Fatal.Injuries']/purpose_df['total_pop']) * 100
purpose_df['injury_perc'] = ((purpose_df['Total.Serious.Injuries']+purpose_df['Total.Minor.Injuries'])/purpose_df['total_pop']) * 100
purpose_df['uninjured_perc'] = (purpose_df['Total.Uninjured']/purpose_df['total_pop']) * 100
purpose_df.reset_index(inplace = True)

In [45]:
purpose_df = purpose_df[purpose_df['Number.of.Engines']!=-99]

In [46]:
purpose_df.drop(['Total.Fatal.Injuries', 'Total.Serious.Injuries','Total.Minor.Injuries', 'Total.Uninjured' ], axis=1, inplace=True)

In [47]:
purpose_df.sort_values('uninjured_perc', ascending = False).head(10)

,Make,Number.of.Engines,Engine.Type,Purpose.of.flight,year,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
6,Airbus,2.0,Unknown,Unknown,78736,39,3079.0,0.000000,0.909386,99.090614
30,Bombardier,2.0,Turbo Fan,Unknown,66460,33,1930.0,0.000000,1.036269,98.963731
5,Airbus,2.0,Turbo Fan,Unknown,151188,75,8997.0,0.022230,1.200400,98.777370
57,Embraer,2.0,Turbo Fan,Unknown,54385,27,1479.0,0.270453,1.217039,98.512508
28,Boeing,2.0,Unknown,Unknown,260211,129,10740.0,1.247672,0.726257,98.026071
27,Boeing,2.0,Turbo Fan,Unknown,354680,176,19924.0,0.035134,2.489460,97.475407
1,Air Tractor,1.0,Reciprocating,Aerial Application,62439,31,31.0,3.225806,12.903226,83.870968
18,Beech,2.0,Turbo Prop,Unknown,76562,38,181.0,12.707182,7.734807,79.558011
63,Maule,1.0,Reciprocating,Personal,185438,92,138.0,5.797101,15.217391,78.985507
37,Cessna,1.0,Reciprocating,Instructional,1419195,704,1129.0,5.225864,16.031887,78.742250


In [48]:
purpose_df2 = purpose_df.sort_values('uninjured_perc', ascending = False).head(10)

In [49]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

styled_table = (purpose_df2.style
    .format({'Total.Fatal.Injuries': '{:,.0f}', 'Total.Serious.Injuries': '{:,.0f}', 'Total.Minor.Injuries': '{:,.0f}', 'Total.Uninjured': '{:,.0f}', 'total_pop': '{:,.0f}', 'fatality_perc': '{:.2f}%', 'injury_perc': '{:.2f}%', 'uninjured_perc': '{:.2f}%'})
    .format({'Number.of.Engines': '{:.0f}'})
    .applymap(lambda x: 'background-color: lightgreen' if isinstance(x, float) and x > 90.00 else ('background-color: lightcoral' if x == -99 else ''), subset=pd.IndexSlice[:, ['uninjured_perc', 'Number.of.Engines']])
    .set_properties(**{'font-size': '12pt', 'font-family': 'Calibri', 'text-align': 'center', 'border': '1px solid black'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#337ab7'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid grey')]}])
                        .hide_index())
    
display(styled_table)

Make,Number.of.Engines,Engine.Type,Purpose.of.flight,year,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
Airbus,2,Unknown,Unknown,78736,39,"3,079",0.00%,0.91%,99.09%
Bombardier,2,Turbo Fan,Unknown,66460,33,"1,930",0.00%,1.04%,98.96%
Airbus,2,Turbo Fan,Unknown,151188,75,"8,997",0.02%,1.20%,98.78%
Embraer,2,Turbo Fan,Unknown,54385,27,"1,479",0.27%,1.22%,98.51%
Boeing,2,Unknown,Unknown,260211,129,"10,740",1.25%,0.73%,98.03%
Boeing,2,Turbo Fan,Unknown,354680,176,"19,924",0.04%,2.49%,97.48%
Air Tractor,1,Reciprocating,Aerial Application,62439,31,31,3.23%,12.90%,83.87%
Beech,2,Turbo Prop,Unknown,76562,38,181,12.71%,7.73%,79.56%
Maule,1,Reciprocating,Personal,185438,92,138,5.80%,15.22%,78.99%
Cessna,1,Reciprocating,Instructional,1419195,704,"1,129",5.23%,16.03%,78.74%


In [50]:
purpose_df[purpose_df['Make']=='Airbus']

,Make,Number.of.Engines,Engine.Type,Purpose.of.flight,year,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
5,Airbus,2.0,Turbo Fan,Unknown,151188,75,8997.0,0.02223,1.200400,98.777370
6,Airbus,2.0,Unknown,Unknown,78736,39,3079.0,0.00000,0.909386,99.090614


In [51]:
purpose_df3 = purpose_df[purpose_df['Make']=='Airbus']

In [52]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

styled_table = (purpose_df3.style
    .format({'Total.Fatal.Injuries': '{:,.0f}', 'Total.Serious.Injuries': '{:,.0f}', 'Total.Minor.Injuries': '{:,.0f}', 'Total.Uninjured': '{:,.0f}', 'total_pop': '{:,.0f}', 'fatality_perc': '{:.2f}%', 'injury_perc': '{:.2f}%', 'uninjured_perc': '{:.2f}%'})
    .format({'Number.of.Engines': '{:.0f}'})
    .applymap(lambda x: 'background-color: lightgreen' if isinstance(x, float) and x > 90.00 else ('background-color: lightcoral' if x == -99 else ''), subset=pd.IndexSlice[:, ['uninjured_perc', 'Number.of.Engines']])
    .set_properties(**{'font-size': '12pt', 'font-family': 'Calibri', 'text-align': 'center', 'border': '1px solid black'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#337ab7'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid grey')]}])
                        .hide_index())
    
display(styled_table)

# fig=styled_table.get_figure()
# plt.savefig('table.png')





Make,Number.of.Engines,Engine.Type,Purpose.of.flight,year,counter,total_pop,fatality_perc,injury_perc,uninjured_perc
Airbus,2,Turbo Fan,Unknown,151188,75,"8,997",0.02%,1.20%,98.78%
Airbus,2,Unknown,Unknown,78736,39,"3,079",0.00%,0.91%,99.09%


In [53]:
styled_table.to_csv('filename.csv', index=False)

AttributeError: 'Styler' object has no attribute 'to_csv'

# Grouping by Make, number of eng, eng type, and model

In [ ]:
model_df = df_edited2.groupby(['Make','Number.of.Engines','Engine.Type', 'Model']).sum()

In [ ]:
model_df = model_df[model_df['counter'] >= 32]

In [ ]:
model_df['fatality_perc'] = (model_df['Total.Fatal.Injuries']/model_df['total_pop']) * 100
model_df['injury_perc'] = ((model_df['Total.Serious.Injuries']+model_df['Total.Minor.Injuries'])/model_df['total_pop']) * 100
model_df['uninjured_perc'] = (model_df['Total.Uninjured']/model_df['total_pop']) * 100
model_df.reset_index(inplace = True)

In [ ]:
model_df = model_df[model_df['Number.of.Engines']!=-99]

In [ ]:
model_df.sort_values('uninjured_perc', ascending = False).head(20)

In [ ]:
model_df1 = model_df.sort_values('uninjured_perc', ascending = False).head(20)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

styled_table = (model_df1.style
    .format({'Total.Fatal.Injuries': '{:,.0f}', 'Total.Serious.Injuries': '{:,.0f}', 'Total.Minor.Injuries': '{:,.0f}', 'Total.Uninjured': '{:,.0f}', 'total_pop': '{:,.0f}', 'fatality_perc': '{:.2f}%', 'injury_perc': '{:.2f}%', 'uninjured_perc': '{:.2f}%'})
    .format({'Number.of.Engines': '{:.0f}'})
    .applymap(lambda x: 'background-color: lightgreen' if isinstance(x, float) and x > 90.00 else ('background-color: lightcoral' if x == -99 else ''), subset=pd.IndexSlice[:, ['uninjured_perc', 'Number.of.Engines']])
    .set_properties(**{'font-size': '12pt', 'font-family': 'Calibri', 'text-align': 'center', 'border': '1px solid black'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#337ab7'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid grey')]}])
                        .hide_index())



display(styled_table)

# Group by Make,  model, purpose

In [ ]:
new_df = df_edited2.groupby(['Make', 'Model', 'Purpose.of.flight']).sum()

In [ ]:
new_df = new_df[new_df['counter'] >= 32]

In [ ]:
new_df['fatality_perc'] = (new_df['Total.Fatal.Injuries']/new_df['total_pop']) * 100
new_df['injury_perc'] = ((new_df['Total.Serious.Injuries']+new_df['Total.Minor.Injuries'])/new_df['total_pop']) * 100
new_df['uninjured_perc'] = (new_df['Total.Uninjured']/new_df['total_pop']) * 100
new_df.reset_index(inplace = True)

In [ ]:
new_df.drop(['Number.of.Engines'], axis = 1, inplace = True)

In [ ]:
new_df.sort_values('uninjured_perc', ascending = False).head(20)

In [ ]:
new_df.sort_values('fatality_perc', ascending = False).head(20)

# Group by Make and Model (Malaysia 737 bad)

In [ ]:
model_make = df_edited2.groupby(['Make', 'Model']).sum()

In [ ]:
model_make = model_make[model_make['counter'] >= 10]

In [ ]:
model_make['walkaway_numb'] = model_make['Total.Minor.Injuries'] + model_make['Total.Uninjured']

In [ ]:
model_make['fatality_perc'] = (model_make['Total.Fatal.Injuries']/model_make['total_pop']) * 100
model_make['injury_perc'] = ((model_make['Total.Serious.Injuries']+model_make['Total.Minor.Injuries'])/model_make['total_pop']) * 100
model_make['uninjured_perc'] = (model_make['Total.Uninjured']/model_make['total_pop']) * 100
model_make.reset_index(inplace = True)

In [ ]:
model_make['walkaway_perc'] = ((model_make['Total.Minor.Injuries'] + model_make['Total.Uninjured']) / model_make['total_pop'] ) * 100

In [ ]:
model_make.drop(['Number.of.Engines'], axis = 1, inplace = True)

In [ ]:
model_make.drop(['year'], axis = 1, inplace = True)

In [ ]:
model_make = model_make.drop(['walkaway_perc'], axis=1)

In [ ]:
model_make.sort_values('uninjured_perc', ascending = False).head(20)

In [ ]:
model_make = model_make.sort_values('uninjured_perc', ascending = False).head(20)

In [ ]:
model_make.drop(['Total.Fatal.Injuries', 'Total.Serious.Injuries','Total.Minor.Injuries', 'Total.Uninjured' ], axis=1, inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

styled_table = (model_make.style
    .format({'Total.Fatal.Injuries': '{:,.0f}', 'Total.Serious.Injuries': '{:,.0f}', 'Total.Minor.Injuries': '{:,.0f}', 'Total.Uninjured': '{:,.0f}', 'total_pop': '{:,.0f}', 'fatality_perc': '{:.2f}%', 'injury_perc': '{:.2f}%', 'uninjured_perc': '{:.2f}%'})
    .applymap(lambda x: 'background-color: lightgreen' if isinstance(x, float) and x >= 90.00 else ('background-color: lightcoral' if x == -99 else ''), subset=pd.IndexSlice[:, ['uninjured_perc']])
    .set_properties(**{'font-size': '12pt', 'font-family': 'Calibri', 'text-align': 'center', 'border': '1px solid black'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#337ab7'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center')]},
                       {'selector': 'tbody tr', 'props': [('border-bottom', '1px solid grey')]}])
                        .hide_index())



display(styled_table)

In [ ]:
model_make5 = df_edited2.groupby(['Make', 'Model', 'Number.of.Engines']).sum()

In [ ]:
model_make5.head(20)

In [ ]:
model_make.sort_values('fatality_perc', ascending = False).head(20)
#IF CRASH, 75% die in PIPER PA25

In [ ]:
model_make['counter'].sum()

# Weather IMC vs VMC

In [ ]:
weather_cond_val = df_edited2['Weather.Condition'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))



ax.bar(x = weather_cond_val.index, height = weather_cond_val.values)



# Questions to ask

- Is cost a factor?